In [42]:
# !pip install crewai crewai-tools langchain-community langchain-openai duckduckgo-search yfinance

In [1]:
# these three lines swap the stdlib sqlite3 lib with the pysqlite3 package
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# https://github.com/crewAIInc/crewAI/issues/2881#issuecomment-2904332609
# from rich.console import Console
# import sys
# import builtins

# # Create a safe console for Jupyter (no ANSI, no recursion)
# console = Console(force_terminal=False, file=sys.__stdout__)

# # Optional: make all `print()` use the safe console
# builtins.print = console.print

In [ ]:
# imports
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from crewai_tools import CSVSearchTool # RAG for CSV
import os
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = ""
llm = ChatOpenAI(model="gpt-4.1-mini")

In [4]:
# Search tool
from langchain_community.tools import DuckDuckGoSearchResults

searchTool = DuckDuckGoSearchResults(backend='api', num_results=10)

@tool
def search(query: str):
    """Searches information using DuckDuckGo."""
    return searchTool.invoke(query)

In [5]:
videoScriptwriter = Agent(
    role="Video Scriptwriter specialized in YouTube videos",
    goal="""Create a detailed, engaging, and structured script tailored for a full-length YouTube content, including hooks, storytelling, transitions, and calls to action.""",
    backstory="""You're a highly experienced in writing scripts for gaming YouTube videos.
        You also have a large experience in overall content creation.
        You are a storytelling specialist when it comes to gaming videos.
        You always search for new gaming content to make sure your gaming knowledge is up-to-date, always considering the source of the searched content.""",
    llm=llm,
    max_iter=5,
    allow_delegation=False, # can't delegate his task to another agent
    verbose=False,
    memory=True,
    tools=[search]
)
writeYoutubeVideoScript = Task(
    description="""Write a detailed, engaging, and structure script tailored for a full-length YouTube video script for {query}.
    Include hooks, storytelling, transitions, and calls to action.
    Make sure your knowledge regarding {query} is up-to-date by searching it from trusted sources such as official game websites, major gaming news outlets (e.g., IGN, Kotaku), or well-known community wikis.""",
    expected_output="""A well-written, structured, detailed YouTube video script for the proposed theme.""",
    agent=videoScriptwriter
)

# suggestion from GPT - not doing it for now
# expected_output="""A structured script using the following format:

# - Title: ...
# - Hook (0:00–0:30): ...
# - Intro (0:30–1:00): ...
# - Main Storyline (1:00–X): ...
# - Climax / Twist: ...
# - CTA (Call to Action): ...
# - Outro: ...

# The script should have timestamps, speaker tone suggestions (if relevant), and transitions clearly marked."""

In [6]:
thumbnailCreator = Agent(
    role="Thumbnail creator specialist",
    goal="""Create thumbnails inspired in a given YouTube video content, remarking attractive visual elements.""",
    backstory="""You are an expert in creating thumbnails for a variety of YouTube videos, always taking into account the video script and content.
    You create impressive thumbnails using visual elements that sparks curiosity in the viewers.""",
    llm=llm,
    max_iter=5,
    allow_delegation=True,
    verbose=False,
    memory=True,
    tools=[search]
)
createThumbnails = Task(
    description="""Create three variations of thumbnails for the YouTube video script using its content as an inspiration.
    If the video script lacks clarity or inspiration for thumbnail creation, you may delegate the task back to the original `videoScriptwriter` agent to revise or enrich the script content.""",
    expected_output="""Three thumbnail concepts with the following format:
    - Suggested title text (short, 3 to 6 words)
    - Description of imagery and layout (what's shown, colors, expressions, etc.)
    - Emotions or curiosity triggers used (e.g., surprise, suspense, humor)
    - Optional: callouts or design notes (e.g., “add red arrow pointing to treasure chest”)""",
    agent=thumbnailCreator,
    context=[writeYoutubeVideoScript]
)

In [7]:
videoScriptReviewer = Agent(
    role="Senior video script reviewer",
    goal="""Review the video script to ensure it is well-structured, has detailed content, is ready to be recorded, and has thumbnail options that matches the video content.""",
    backstory="""You are an unbelievable video script reviewer that understands what a high-quality YouTube video should have, and how it should be structured.
    "You always know when a thumbnail is appropriate for the video content, and if it will make the audience curious.""",
    llm=llm,
    max_iter=5,
    allow_delegation=True,
    verbose=False,
    memory=True
)
review = Task(
    description="""Review the YouTube video script and the associated thumbnails to write the final YouTube video script.
    Include the three thumbnail options, and write a brief summary on why each one should be used.
    If you think the video script and/or the thumbnails aren't good enough, you can delegate back to the video script writer and/or the thumbnail creator so they work again on their task.
    If a task is delegated, provide a specific comment explaining *what must be improved* (e.g., "script lacks a strong hook," or "thumbnail doesn’t match climax scene").""",
    expected_output="""A final, polished YouTube video script and three thumbnail options, with justifications for each.
        Output format:
        ---
        SCRIPT:
        <final video script content>

        THUMBNAILS:
        1. Thumbnail Concept 1
        - Justification: ...
        2. Thumbnail Concept 2
        - Justification: ...
        3. Thumbnail Concept 3
        - Justification: ...
   """,
    agent=videoScriptReviewer,
    context=[writeYoutubeVideoScript, createThumbnails]
)

In [12]:
crew = Crew(
    agents=[videoScriptwriter, thumbnailCreator, videoScriptReviewer],
    tasks=[writeYoutubeVideoScript, createThumbnails, review],
    verbose=False,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm
)

In [13]:
results = crew.kickoff(inputs={'query': 'Top 5 best games for 2024'})

In [14]:
from IPython.display import Markdown
Markdown(results.raw)

---
SCRIPT:
"Hey gamers! 2024 is shaping up to be one of the most exciting years in gaming history. Whether you love sprawling open worlds, dark fantasy adventures, or spine-chilling horror, this year has something epic in store for you. Today, we’re counting down the Top 5 Best Games to watch for in 2024. So grab your controller, sit back, and let’s dive right in!

From magical expansions to brand-new universes, these games promise to deliver unforgettable experiences that will keep you hooked for hours. Don’t forget to hit that like button and subscribe for more awesome gaming content. Now, let’s jump into our list!

Starting off our list at number five is the much-anticipated Dark Arts Expansion for Hogwarts Legacy. After the massive success of the original game, fans are eager to dive deeper into the wizarding world, but this time exploring the darker side of magic. Imagine mastering forbidden spells and uncovering hidden secrets in the shadowy corners of Hogwarts. With beautiful next-gen graphics and a rich narrative, this expansion is a must-play for any Harry Potter fan or RPG lover. It’s coming to PlayStation 5, Xbox Series X|S, and PC this March, and IGN couldn’t stop praising its immersive storytelling.

From the enchanted halls of Hogwarts to the vast expanse of space, our next game takes us on a journey beyond the stars…

At number four, we have Starfield, Bethesda’s first new IP in over 25 years. This sci-fi RPG lets you explore an expansive galaxy filled with mysteries and adventure. Whether you want to customize your spaceship, forge alliances, or uncover ancient secrets, Starfield offers unparalleled freedom in an ambitious universe. Launched on Xbox Series X|S and PC in February, GameSpot calls it a game-changer for sci-fi fans and RPG enthusiasts alike. If you dream of space exploration, this title is your launchpad.

Speaking of epic journeys, Final Fantasy fans have a new adventure to look forward to…

Coming in at number three is Final Fantasy XVI, a visually stunning and narratively rich action RPG exclusive to PlayStation 5. Known for its intricate storylines and innovative combat, this latest installment promises a mature tale filled with political intrigue, powerful summons, and breathtaking battles. Set for release in June, both IGN and Kotaku have praised its fresh take on the beloved franchise, making it a must-play for fans and newcomers alike.

Next up, we head to a legendary franchise that continues to redefine adventure on Nintendo Switch…

At number two is The Legend of Zelda: Echoes of Time, the highly anticipated new installment for the Nintendo Switch. Building on the groundbreaking success of Breath of the Wild, this game is expected to introduce innovative gameplay mechanics and a deeper story that will captivate both longtime fans and new players. Although we’re still awaiting official details, trusted outlets like IGN and Kotaku expect it to be one of the defining Switch games of 2024.

And now, for our number one pick—prepare yourself for a terrifyingly good time…

Topping our list is the Resident Evil 4 Remake, arriving on PlayStation 5, Xbox Series X|S, and PC this March. Capcom has masterfully reimagined this classic survival horror title with stunning visuals, modernized gameplay, and an enhanced story that respects the original while introducing new elements. Known as one of the greatest games of all time, this remake is praised by GameSpot for perfectly balancing nostalgia and innovation. Whether you’re a horror veteran or a newcomer, Resident Evil 4 Remake promises intense thrills and chills.

There you have it—the Top 5 Best Games for 2024 that you absolutely cannot miss. Which one are you most excited to play? Drop your thoughts in the comments below! And if you enjoyed this countdown, please give the video a thumbs up and subscribe for more deep dives into the world of gaming. Thanks for watching, and I’ll see you in the next video!

Check out our playlist for more top game lists and reviews, and don’t forget to hit that notification bell so you never miss an update!"

THUMBNAILS:
1. Thumbnail Concept 1
- Justification: This thumbnail features a dynamic and colorful collage of characters and elements from all five games, instantly conveying the variety and excitement viewers can expect. The vibrant gradient background with glowing effects adds energy and mystery, while the bold title text with a "2024" badge draws immediate attention. This design appeals to fans of the games and sparks curiosity through its detailed imagery.

2. Thumbnail Concept 2
- Justification: Centering on a large game controller surrounded by iconic elements from each game, this thumbnail emphasizes the essential nature of these titles for gamers in 2024. The sleek black background with electric blue and orange streaks injects energy and urgency, while the metallic silver 3D-effect title conveys importance. This option appeals to gamers looking for must-play recommendations and has a modern, polished look.

3. Thumbnail Concept 3
- Justification: Utilizing a countdown timer or scoreboard style, this thumbnail visually highlights the ranking format of the video, engaging viewers with suspense and excitement. The fiery orange and red gradient background invokes intensity and drama, particularly with the bright red highlight on the number 1 slot (Resident Evil 4 Remake). The distressed texture on the bold white title adds an edgy feel. This design effectively communicates the countdown theme and entices viewers interested in rankings and top lists.